<a href="https://colab.research.google.com/github/Akshata4/data_mining/blob/main/apachebeam/apachebeam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>